# Etapa 1 - PyMongo para ingestão de dados

In [1]:
import csv
from pymongo import MongoClient

In [6]:
# Conexão com o MongoDB

client = MongoClient("mongodb://mongo-db:27017")
db = client["ufrn"]
collection = db["discentes_2024"]

collection.delete_many({})

with open("discentes-2024.csv", newline='', encoding="utf-8") as f:
    reader = csv.DictReader(f, delimiter=';')
    documentos = list(reader)
    collection.insert_many(documentos)

print(f"{len(documentos)} documentos inseridos com sucesso no MongoDB")

14459 documentos inseridos com sucesso no MongoDB


In [7]:
for doc in collection.find().limit(5):
    print(doc)

{'_id': ObjectId('68719d64cb8543122036aff0'), 'matricula': '20241004169', 'ano_ingresso': '2024', 'periodo_ingresso': '1', 'id_curso': '96067713', 'id_unidade': '7268', 'id_unidade_gestora': '445', 'nome_discente': 'ABDIAS DA SILVA TAVARES', 'sexo': 'M', 'forma_ingresso': 'SELEÇÃO DE PÓS-GRADUAÇÃO', 'tipo_discente': 'REGULAR', 'status': 'ATIVO', 'sigla_nivel_ensino': 'E', 'nivel_ensino': 'MESTRADO', 'nome_curso': 'MESTRADO PROFISSIONAL EM ENERGIA ELÉTRICA', 'modalidade_educacao': 'PRESENCIAL', 'nome_unidade': 'PROGRAMA DE PÓS-GRADUAÇÃO EM ENERGIA ELÉTRICA', 'nome_unidade_gestora': 'CENTRO DE TECNOLOGIA'}
{'_id': ObjectId('68719d64cb8543122036aff1'), 'matricula': '20241030122', 'ano_ingresso': '2024', 'periodo_ingresso': '2', 'id_curso': '', 'id_unidade': '', 'id_unidade_gestora': '', 'nome_discente': 'ABDIEL DE LIRA ROLIM', 'sexo': 'M', 'forma_ingresso': 'ALUNO ESPECIAL POS-GRADUACAO', 'tipo_discente': 'ESPECIAL', 'status': 'ATIVO', 'sigla_nivel_ensino': 'E', 'nivel_ensino': 'MESTRADO'

# Etapa 2 - PySpark para processamento, exibição e salvamento

In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder \
    .appName("DiscentesUFRN") \
    .config("spark.mongodb.read.connection.uri", "mongodb://mongo-db:27017") \
    .config("spark.mongodb.read.database", "ufrn") \
    .config("spark.mongodb.read.collection", "discentes_2024") \
    .config("spark.mongodb.write.connection.uri", "mongodb://mongo-db:27017") \
    .config("spark.mongodb.write.database", "ufrn") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:10.4.0") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/myuser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/myuser/.ivy2/cache
The jars for the packages stored in: /home/myuser/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a71ae79d-1f36-4b31-a8fc-d6bccefe4227;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.4.0 in central
	found org.mongodb#mongodb-driver-sync;5.1.4 in central
	[5.1.4] org.mongodb#mongodb-driver-sync;[5.1.1,5.1.99)
	found org.mongodb#bson;5.1.4 in central
	found org.mongodb#mongodb-driver-core;5.1.4 in central
	found org.mongodb#bson-record-codec;5.1.4 in central
:: resolution report :: resolve 2480ms :: artifacts dl 3ms
	:: modules in use:
	org.mongodb#bson;5.1.4 from central in [default]
	org.mongodb#bson-record-codec;5.1.4 from central in [default]
	org.mongodb#mongodb-driver-core;5.1.4 from central in [default]
	org.mongodb#mongodb-driver-sync;5.1.4 from central in [default]
	org.mongodb.spark#mongo-spark-connector_

In [10]:
df = spark.read.format("mongodb").load()
df.printSchema()
df.show(5)

root
 |-- _id: string (nullable = true)
 |-- ano_ingresso: string (nullable = true)
 |-- forma_ingresso: string (nullable = true)
 |-- id_curso: string (nullable = true)
 |-- id_unidade: string (nullable = true)
 |-- id_unidade_gestora: string (nullable = true)
 |-- matricula: string (nullable = true)
 |-- modalidade_educacao: string (nullable = true)
 |-- nivel_ensino: string (nullable = true)
 |-- nome_curso: string (nullable = true)
 |-- nome_discente: string (nullable = true)
 |-- nome_unidade: string (nullable = true)
 |-- nome_unidade_gestora: string (nullable = true)
 |-- periodo_ingresso: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- sigla_nivel_ensino: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tipo_discente: string (nullable = true)



+--------------------+------------+--------------------+---------+----------+------------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+---------+-------------+
|                 _id|ano_ingresso|      forma_ingresso| id_curso|id_unidade|id_unidade_gestora|  matricula|modalidade_educacao|        nivel_ensino|          nome_curso|       nome_discente|        nome_unidade|nome_unidade_gestora|periodo_ingresso|sexo|sigla_nivel_ensino|   status|tipo_discente|
+--------------------+------------+--------------------+---------+----------+------------------+-----------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+---------+-------------+
|68719d64cb8543122...|        2024|SELEÇÃO DE PÓS-GR...| 96067713|      7268|               445|2024100

In [11]:
df.createOrReplaceTempView("discentes")

## Consultas

### 1. Listar todos os alunos que ingressaram por meio do SiSU

In [9]:
sisu_df = df.filter(df["forma_ingresso"] == "SiSU")
sisu_df.show(5, truncate=False)

+------------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+------------------------+--------------------------------------------+------------------------------------+-------------------------------------------+----------------+----+------------------+---------+-------------+
|_id                     |ano_ingresso|forma_ingresso|id_curso|id_unidade|id_unidade_gestora|matricula  |modalidade_educacao|nivel_ensino|nome_curso              |nome_discente                               |nome_unidade                        |nome_unidade_gestora                       |periodo_ingresso|sexo|sigla_nivel_ensino|status   |tipo_discente|
+------------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+------------------------+--------------------------------------------+------------------------------------+-------------------------------------------+-

In [10]:
#Salvar resultado
sisu_df.write.format("mongodb").mode("overwrite") \
    .option("collection", "ingresso_sisu").save()

### 2. Computar quantos alunos são do sexo masculino e do sexo feminino

In [11]:
sexo_df = df.groupBy("sexo").count()
sexo_df.show()

+-----+-----+
| sexo|count|
+-----+-----+
|false|    1|
|    F| 7470|
|    M| 6988|
+-----+-----+



In [12]:
df.filter(df["sexo"] == "false").show(truncate=False)

+------------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+------------------+------------------+--------------------+--------------------+----------------+-----+------------------+------+-------------+
|_id                     |ano_ingresso|forma_ingresso|id_curso|id_unidade|id_unidade_gestora|matricula  |modalidade_educacao|nivel_ensino|nome_curso        |nome_discente     |nome_unidade        |nome_unidade_gestora|periodo_ingresso|sexo |sigla_nivel_ensino|status|tipo_discente|
+------------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+------------------+------------------+--------------------+--------------------+----------------+-----+------------------+------+-------------+
|68719d64cb8543122036c8bc|2024        |SiSU          |2000032 |445       |445               |20240008299|PRESENCIAL         |GRADUAÇÃO   |ENGENHARIA QUÍMI

In [13]:
df_new = df.filter(df["sexo"].isin("M", "F"))

sexo_df = df_new.groupBy("sexo").count()
sexo_df.show()

+----+-----+
|sexo|count|
+----+-----+
|   F| 7470|
|   M| 6988|
+----+-----+



In [13]:
# Salvando consulta
sexo_df.write.format("mongodb").mode("overwrite") \
    .option("collection", "contagem_por_sexo").save()

### 3. Computar o top 5 dos cursos que mais receberam alunos

In [14]:
top_cursos_df = df.groupBy("nome_curso") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(5)

top_cursos_df.show(truncate=False)

+--------------------------------+-----+
|nome_curso                      |count|
+--------------------------------+-----+
|                                |1260 |
|CIÊNCIAS E TECNOLOGIA           |787  |
|CURSO DE LÍNGUA INGLESA  BÁSICO|494  |
|PEDAGOGIA                       |427  |
|TECNOLOGIA DA INFORMAÇÃO        |346  |
+--------------------------------+-----+



In [15]:
from pyspark.sql.functions import col

In [16]:
df.filter(col("nome_curso").rlike("^\s*$")).show(truncate=False)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3249/398303428.py:1: SyntaxWarning: invalid escape sequence '\s'
  df.filter(col("nome_curso").rlike("^\s*$")).show(truncate=False)


+------------------------+------------+----------------------------+--------+----------+------------------+-----------+-------------------+------------+----------+--------------------------------------+------------+--------------------+----------------+----+------------------+---------+-------------+
|_id                     |ano_ingresso|forma_ingresso              |id_curso|id_unidade|id_unidade_gestora|matricula  |modalidade_educacao|nivel_ensino|nome_curso|nome_discente                         |nome_unidade|nome_unidade_gestora|periodo_ingresso|sexo|sigla_nivel_ensino|status   |tipo_discente|
+------------------------+------------+----------------------------+--------+----------+------------------+-----------+-------------------+------------+----------+--------------------------------------+------------+--------------------+----------------+----+------------------+---------+-------------+
|68719d64cb8543122036aff1|2024        |ALUNO ESPECIAL POS-GRADUACAO|        |          |      

In [17]:
# Remover cursos com nome vazio ou espaços
df_validos = df.filter(~col("nome_curso").rlike("^\s*$"))

# Computar top 5 cursos
top5_df = df_validos.groupBy("nome_curso") \
    .count() \
    .orderBy("count", ascending=False) \
    .limit(5)

# Mostrar resultado
top5_df.show(truncate=False)

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3249/1766389417.py:2: SyntaxWarning: invalid escape sequence '\s'
  df_validos = df.filter(~col("nome_curso").rlike("^\s*$"))


+--------------------------------+-----+
|nome_curso                      |count|
+--------------------------------+-----+
|CIÊNCIAS E TECNOLOGIA           |787  |
|CURSO DE LÍNGUA INGLESA  BÁSICO|494  |
|PEDAGOGIA                       |427  |
|TECNOLOGIA DA INFORMAÇÃO        |346  |
|HISTÓRIA                        |309  |
+--------------------------------+-----+



In [18]:
top5_df.write.format("mongodb").mode("overwrite") \
    .option("collection", "top5_cursos").save()

### 4. quantos alunos são do sexo masculino que ingressaram via SiSU em Engenharia de Computação.

In [19]:
# Limpar dados inválidos
df_limpo = df.filter(col("sexo").isin("M", "F")) \
             .filter(~col("nome_curso").rlike("^\s*$"))

#Filtrar apenas os homens, via SiSU, em Engenharia de Computação
engenharia_df = df_limpo.filter(
    (col("sexo") == "M") &
    (col("forma_ingresso") == "SiSU") &
    (col("nome_curso") == "ENGENHARIA DE COMPUTAÇÃO")
)

# Mostrar resultado
engenharia_df.show(truncate=False)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_3249/1300903563.py:3: SyntaxWarning: invalid escape sequence '\s'
  .filter(~col("nome_curso").rlike("^\s*$"))


+------------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+------------------------+----------------------------------------------+--------------------+--------------------+----------------+----+------------------+--------+-------------+
|_id                     |ano_ingresso|forma_ingresso|id_curso|id_unidade|id_unidade_gestora|matricula  |modalidade_educacao|nivel_ensino|nome_curso              |nome_discente                                 |nome_unidade        |nome_unidade_gestora|periodo_ingresso|sexo|sigla_nivel_ensino|status  |tipo_discente|
+------------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+------------------------+----------------------------------------------+--------------------+--------------------+----------------+----+------------------+--------+-------------+
|68719d64cb8543122036b0af|2024        |SiSU      

In [20]:
total = engenharia_df.count()
print(f"\nTotal de homens via SiSU em Engenharia de Computação: {total}")


Total de homens via SiSU em Engenharia de Computação: 32


In [21]:
engenharia_df.write.format("mongodb").mode("overwrite") \
    .option("collection", "homens_sisu_engenharia_computacao").save()

## Verificando as coleções salvas

In [22]:
print("Verificando a coleção 'ingresso_sisu':")
spark.read.format("mongodb") \
    .option("collection", "ingresso_sisu") \
    .load() \
    .show()

Verificando a coleção 'ingresso_sisu':
++
||
++
++



25/07/11 23:27:16 WARN Partitioner: Unable to get collection stats (collstats) returning a single partition.
25/07/11 23:27:16 WARN Partitioner: Unable to get collection stats (collstats) returning a single partition.


In [23]:
print("Verificando a coleção 'contagem_por_sexo':")
spark.read.format("mongodb") \
    .option("collection", "contagem_por_sexo") \
    .load() \
    .show()

Verificando a coleção 'contagem_por_sexo':
++
||
++
++



25/07/11 23:27:18 WARN Partitioner: Unable to get collection stats (collstats) returning a single partition.
25/07/11 23:27:18 WARN Partitioner: Unable to get collection stats (collstats) returning a single partition.


In [24]:
print("Verificando a coleção 'top5_cursos':")
spark.read.format("mongodb") \
    .option("collection", "top5_cursos") \
    .load() \
    .show()

Verificando a coleção 'top5_cursos':
+--------------------+-----+--------------------+
|                 _id|count|          nome_curso|
+--------------------+-----+--------------------+
|68719dbe2189d1470...|  787|CIÊNCIAS E TECNOL...|
|68719dbe2189d1470...|  494|CURSO DE LÍNGUA I...|
|68719dbe2189d1470...|  427|           PEDAGOGIA|
|68719dbe2189d1470...|  346|TECNOLOGIA DA INF...|
|68719dbe2189d1470...|  309|            HISTÓRIA|
+--------------------+-----+--------------------+



In [25]:
print("Verificando a coleção 'homens_sisu_engenharia_computacao':")
spark.read.format("mongodb") \
    .option("collection", "homens_sisu_engenharia_computacao") \
    .load() \
    .show()

Verificando a coleção 'homens_sisu_engenharia_computacao':
+--------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+--------+-------------+
|                 _id|ano_ingresso|forma_ingresso|id_curso|id_unidade|id_unidade_gestora|  matricula|modalidade_educacao|nivel_ensino|          nome_curso|       nome_discente|        nome_unidade|nome_unidade_gestora|periodo_ingresso|sexo|sigla_nivel_ensino|  status|tipo_discente|
+--------------------+------------+--------------+--------+----------+------------------+-----------+-------------------+------------+--------------------+--------------------+--------------------+--------------------+----------------+----+------------------+--------+-------------+
|68719d64cb8543122...|        2024|          SiSU| 2000026|       445|               445|202

In [ ]:
#Exemplo usado para ser monitorado pelo kafka
from pymongo import MongoClient

mongo_service_name = "mongo-db"
mongo_uri = f"mongodb://{mongo_service_name}:27017/"

try:
    client = MongoClient(mongo_uri)
    db = client["ufrn"] 
    collection = db["top5_cursos"]
    # Dados a serem inseridos
    new_doc = {
        "curso": "yyyyyyyyyyyyyyyyyyyyyyyyyy",
        "total_matriculas": 30000
    }

    # Inserir o documento
    result = collection.insert_one(new_doc)
    print(f"Documento inserido com sucesso! ID: {result.inserted_id}")

except Exception as e:
    print(f"Erro ao inserir documento no MongoDB: {e}")
finally:
    if 'client' in locals() and client:
        client.close()


Documento inserido com sucesso! ID: 6871a88fcb8543122036e874
